# Análisis financiero del indice S&P500
### Por Francisco Daniel Lugo

# Visión general
### Este análisis financiero pretende develar los secretos que se ocultan tras el índice SP500 y así tener un criterio más amplio al momento de invertir.

In [44]:
import numpy as np
import pandas as pd
import yfinance as yf
import wikipedia as wp
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
import ssl
import urllib.request
import cufflinks as cf
import plotly.offline as py
import plotly.express as px
import cufflinks as cf
%matplotlib inline

# Make Plotly work in your Jupyter Notebook
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
# Use Plotly locally
cf.go_offline()

## Obtener y limpiar dataset

In [2]:
ssl._create_default_https_context = ssl._create_unverified_context
urllib.request.urlopen("https://en.wikipedia.org/wiki/List_of_S%26P_500_companies")

In [3]:
wiki_data = pd.read_html("https://en.wikipedia.org/wiki/List_of_S%26P_500_companies")
wiki_data = wiki_data[0]
wiki_data

,Symbol,Security,SEC filings,GICS Sector,GICS Sub-Industry,Headquarters Location,Date first added,CIK,Founded
0,MMM,3M,reports,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1976-08-09,66740,1902
1,AOS,A. O. Smith,reports,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916
2,ABT,Abbott,reports,Health Care,Health Care Equipment,"North Chicago, Illinois",1964-03-31,1800,1888
3,ABBV,AbbVie,reports,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
4,ABMD,Abiomed,reports,Health Care,Health Care Equipment,"Danvers, Massachusetts",2018-05-31,815094,1981
...,...,...,...,...,...,...,...,...,...
498,YUM,Yum! Brands,reports,Consumer Discretionary,Restaurants,"Louisville, Kentucky",1997-10-06,1041061,1997
499,ZBRA,Zebra Technologies,reports,Information Technology,Electronic Equipment & Instruments,"Lincolnshire, Illinois",2019-12-23,877212,1969
500,ZBH,Zimmer Biomet,reports,Health Care,Health Care Equipment,"Warsaw, Indiana",2001-08-07,1136869,1927
501,ZION,Zions Bancorporation,reports,Financials,Regional Banks,"Salt Lake City, Utah",2001-06-22,109380,1873


In [4]:
wiki_data = wiki_data[wiki_data.Symbol != "CEG"]
wiki_data.shape


(502, 9)

In [5]:
gics_sector = wiki_data["GICS Sector"].values.tolist()
wiki_data["Symbol"].replace({"BF.B":"BF-B","BRK.B":"BRK-B"}, inplace=True)
indexes = wiki_data.Symbol.values.tolist()
print(len(indexes),len(gics_sector))

502 502


In [6]:
map_dict = {indexes[i]:gics_sector[i] for i in range(0,len(indexes))}
map_dict

{'MMM': 'Industrials',
 'AOS': 'Industrials',
 'ABT': 'Health Care',
 'ABBV': 'Health Care',
 'ABMD': 'Health Care',
 'ACN': 'Information Technology',
 'ATVI': 'Communication Services',
 'ADM': 'Consumer Staples',
 'ADBE': 'Information Technology',
 'ADP': 'Information Technology',
 'AAP': 'Consumer Discretionary',
 'AES': 'Utilities',
 'AFL': 'Financials',
 'A': 'Health Care',
 'APD': 'Materials',
 'AKAM': 'Information Technology',
 'ALK': 'Industrials',
 'ALB': 'Materials',
 'ARE': 'Real Estate',
 'ALGN': 'Health Care',
 'ALLE': 'Industrials',
 'LNT': 'Utilities',
 'ALL': 'Financials',
 'GOOGL': 'Communication Services',
 'GOOG': 'Communication Services',
 'MO': 'Consumer Staples',
 'AMZN': 'Consumer Discretionary',
 'AMCR': 'Materials',
 'AMD': 'Information Technology',
 'AEE': 'Utilities',
 'AAL': 'Industrials',
 'AEP': 'Utilities',
 'AXP': 'Financials',
 'AIG': 'Financials',
 'AMT': 'Real Estate',
 'AWK': 'Utilities',
 'AMP': 'Financials',
 'ABC': 'Health Care',
 'AME': 'Industria

In [7]:
df_list = []
for ticker in indexes:
    data = yf.download(ticker, group_by="Ticker",start="2000-01-01",end="2021-12-31")
    data.reset_index(inplace=True)
    data['Ticker'] = ticker  # I added this column because the df doesn't contain a column with the ticker
    df_list.append(data)



[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [8]:
# combine all dataframes into a single dataframe
sp500 = pd.concat(df_list,ignore_index=True)
sp500.shape

(2478167, 8)

In [9]:
sp500.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2478167 entries, 0 to 2478166
Data columns (total 8 columns):
 #   Column     Dtype         
---  ------     -----         
 0   Date       datetime64[ns]
 1   Open       float64       
 2   High       float64       
 3   Low        float64       
 4   Close      float64       
 5   Adj Close  float64       
 6   Volume     float64       
 7   Ticker     object        
dtypes: datetime64[ns](1), float64(6), object(1)
memory usage: 151.3+ MB


In [10]:
sp500.dropna(inplace=True)
sp500.shape

(2478155, 8)

In [11]:
sp500["Day"] = sp500["Date"].dt.day_name()
sp500["Month"] = sp500["Date"].dt.month_name()
sp500["Year"] = sp500["Date"].dt.year
sp500.head(10)

,Date,Open,High,Low,Close,Adj Close,Volume,Ticker,Day,Month,Year
0,2000-01-03,48.03125,48.25000,47.03125,47.18750,26.434864,2173400.0,MMM,Monday,January,2000
1,2000-01-04,46.43750,47.40625,45.31250,45.31250,25.384462,2713800.0,MMM,Tuesday,January,2000
2,2000-01-05,45.56250,48.12500,45.56250,46.62500,26.119738,3699400.0,MMM,Wednesday,January,2000
3,2000-01-06,47.15625,51.25000,47.15625,50.37500,28.220533,5975800.0,MMM,Thursday,January,2000
4,2000-01-07,50.56250,51.90625,49.96875,51.37500,28.780741,4101200.0,MMM,Friday,January,2000
5,2000-01-10,50.21875,51.75000,50.00000,51.12500,28.640690,3863800.0,MMM,Monday,January,2000
6,2000-01-11,50.37500,51.25000,50.25000,50.25000,28.150503,2357600.0,MMM,Tuesday,January,2000
7,2000-01-12,50.96875,51.81250,50.37500,50.37500,28.220533,2868400.0,MMM,Wednesday,January,2000
8,2000-01-13,50.65625,50.93750,50.18750,50.37500,28.220533,2244400.0,MMM,Thursday,January,2000
9,2000-01-14,50.37500,50.46875,49.46875,49.65625,27.817862,2541800.0,MMM,Friday,January,2000


In [12]:
sp500["GICS sector"] = sp500.Ticker.map(map_dict)
sp500.head(5)

,Date,Open,High,Low,Close,Adj Close,Volume,Ticker,Day,Month,Year,GICS sector
0,2000-01-03,48.03125,48.25000,47.03125,47.1875,26.434864,2173400.0,MMM,Monday,January,2000,Industrials
1,2000-01-04,46.43750,47.40625,45.31250,45.3125,25.384462,2713800.0,MMM,Tuesday,January,2000,Industrials
2,2000-01-05,45.56250,48.12500,45.56250,46.6250,26.119738,3699400.0,MMM,Wednesday,January,2000,Industrials
3,2000-01-06,47.15625,51.25000,47.15625,50.3750,28.220533,5975800.0,MMM,Thursday,January,2000,Industrials
4,2000-01-07,50.56250,51.90625,49.96875,51.3750,28.780741,4101200.0,MMM,Friday,January,2000,Industrials


## Se calculan los indicadores en base a ticker y fecha:

In [13]:
#First sort your dataframe by ticker and date.
#Then apply your calculation when consecutive rows have the same ticker.
#Vectorising is more efficient than calculating results one at a time via lambda.

sp500.sort_values(["Ticker","Date"],inplace=True)
sp500.reset_index(drop=True,inplace=True)
sp500.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Ticker,Day,Month,Year,GICS sector
0,2000-01-03,56.330471,56.464592,48.193848,51.502148,44.221344,4674353.0,A,Monday,January,2000,Health Care
1,2000-01-04,48.730328,49.266811,46.316166,47.567955,40.843323,4765083.0,A,Tuesday,January,2000,Health Care
2,2000-01-05,47.389126,47.567955,43.141991,44.617310,38.309811,5758642.0,A,Wednesday,January,2000,Health Care
3,2000-01-06,44.080830,44.349072,41.577251,42.918453,36.851116,2534434.0,A,Thursday,January,2000,Health Care
4,2000-01-07,42.247852,47.165592,42.203148,46.494991,39.922043,2819626.0,A,Friday,January,2000,Health Care


In [14]:
sp500.loc[sp500['Ticker'] == sp500['Ticker'].shift(), 'Intraday gap return'] = np.log(sp500["Close"]/sp500["Open"]).fillna(0)


In [15]:
sp500.loc[sp500['Ticker'] == sp500['Ticker'].shift(), 'Gap return'] = np.log(sp500["Open"]/sp500["Close"].shift(1)).fillna(0)
sp500.loc[sp500['Ticker'] == sp500['Ticker'].shift(), 'Intraday gap return'] = np.log(sp500["Close"]/sp500["Open"]).fillna(0)
sp500.loc[sp500['Ticker'] == sp500['Ticker'].shift(), 'Variations'] = sp500["Adj Close"].pct_change().fillna(0)
sp500.loc[sp500['Ticker'] == sp500['Ticker'].shift(), 'Volatility'] =  sp500["Variations"].rolling(252).std()*252**0.5




In [16]:
sp500.loc[sp500['Ticker'] == sp500['Ticker'].shift(), 'Volatility'] =  sp500["Variations"].rolling(252).std()*252**0.5
#sp500.Volatility.fillna(0,inplace=True)
sp500.isna().sum()



Date                        0
Open                        0
High                        0
Low                         0
Close                       0
Adj Close                   0
Volume                      0
Ticker                      0
Day                         0
Month                       0
Year                        0
GICS sector                 0
Intraday gap return       502
Gap return                502
Variations                502
Volatility             126412
dtype: int64

In [17]:
sp500["Intraday gap return"].fillna(0,inplace=True)
sp500["Variations"].fillna(0,inplace=True)
sp500["Gap return"].fillna(0,inplace=True)
sp500.isna().sum()

Date                        0
Open                        0
High                        0
Low                         0
Close                       0
Adj Close                   0
Volume                      0
Ticker                      0
Day                         0
Month                       0
Year                        0
GICS sector                 0
Intraday gap return         0
Gap return                  0
Variations                  0
Volatility             126412
dtype: int64

In [18]:
sp500.to_csv(r"C:\Users\user\Desktop\daniel\DATA SCIENCE\HENRY\MATERIAL DE CLASES\DS-PI-ProyectoIndividual\DS-PI-ProyectoIndividual\sp500.csv")


## Mejor día para invertir teniendo en cuenta el retorno de los movimiento gap

In [45]:
gap_return_groupby_day = pd.DataFrame(sp500.groupby("Day")["Gap return"].mean()) #Aqui obtengo la suma por dia)
gap_return_groupby_day.sort_values(by="Gap return",inplace=True)

py.init_notebook_mode(connected=True)
fig = px.bar(gap_return_groupby_day,
             x=gap_return_groupby_day.index,
             y="Gap return", color='Gap return',
            color_continuous_scale=px.colors.sequential.solar)
fig.show()

Como muestra el gráfico, el día de la semana que tiene un gap más alto (en promedio) es el Martes, por lo tanto el día previo (Lunes) es el día ideal para invertir en la bolsa, ya que el martes, nuestras acciones valdrán más.

In [21]:
days_gap_return = {}
for day in sp500.Day.unique().tolist():
    
    x = sp500[sp500.Day == day]["Gap return"].values
    
    q3, q1 = np.nanpercentile(np.array(x), [75,25])
    iqr = q3 - q1
    h_outliers, l_outliers = q3+1.5*iqr , q1-1.5*iqr
    
    y =  x[(x >= l_outliers) & (x <= h_outliers)]
    z = pd.Series(y)
    
    days_gap_return.update({day:z})
    
days_gap_return = pd.DataFrame(days_gap_return)
print(days_gap_return.shape)

(459326, 5)


## ¿Cuál es el mejor día para invertir de acuerdo a movimientos intradiarios?

In [46]:
#Cual es el mejor día para invertir teniendo en cuenta el retorno de los movimientos intradiarios
intraday_gap_returns = pd.DataFrame(sp500.groupby("Day")["Intraday gap return"].mean())
intraday_gap_returns.sort_values(by="Intraday gap return",inplace=True)

py.init_notebook_mode(connected=True)
fig = px.bar(intraday_gap_returns,
             x=intraday_gap_returns.index,
             y="Intraday gap return", color='Intraday gap return',
            color_continuous_scale=px.colors.sequential.deep)
fig.show()

De acuerdo al gráfico, el jueves es el día con gaps intradiarios más altos (en promedio) por lo cual es el mejor día para invertir en la bolsa, si de gaps intradiarios hablamos.

## ¿Cuáles son las mejores industrias que pertenecen al SP500 en las cuales se puede invertir?

In [47]:
Variations_group_by_sector = pd.DataFrame(sp500.groupby("GICS sector")["Variations"].sum())
Variations_group_by_sector.sort_values(by="Variations",inplace=True)

py.init_notebook_mode(connected=True)
fig = px.bar(Variations_group_by_sector,
             x=Variations_group_by_sector.index,
             y="Variations", color='Variations',
            color_continuous_scale=px.colors.sequential.matter)
fig.show()

La suma de variaciones porcentuales a lo largo del tiempo, resulta ser mayor en empresas del area tecnologico-informatico, seguido por cuidado de la salud, e industria. Esto significa que varían más positivamente que negativamente, por lo tanto es razonable invertir en estás empresas.

In [26]:
Sector = sp500["GICS sector"].unique().tolist()

gap_return_sector_sum = {}
intraday_gap_sector_sum = {}
variations_sector_sum = {}
volatility_sector_sum = {}

for field in Sector:
    gap = pd.Series(sp500[sp500["GICS sector"] == field].groupby("Year")["Gap return"].sum().sort_index())
    intra = pd.Series(sp500[sp500["GICS sector"] == field].groupby("Year")["Intraday gap return"].sum().sort_index())
    var = pd.Series(sp500[sp500["GICS sector"] == field].groupby("Year")["Variations"].sum().sort_index())
    volt = pd.Series(sp500[sp500["GICS sector"] == field].groupby("Year")["Volatility"].mean().sort_index())
    

    gap_return_sector_sum.update({field:gap})
    intraday_gap_sector_sum.update({field:intra})
    variations_sector_sum.update({field:var})
    volatility_sector_sum.update({field:volt})
                     
print(len(gap_return_sector_sum),len(intraday_gap_sector_sum),
      len(variations_sector_sum),len(volatility_sector_sum))

gap_return_ssum = pd.DataFrame(gap_return_sector_sum)
intraday_gap_ssum = pd.DataFrame(intraday_gap_sector_sum)
variations_ssum = pd.DataFrame(variations_sector_sum)
volatility_ssum = pd.DataFrame(volatility_sector_sum)

11 11 11 11


In [48]:
py.init_notebook_mode(connected=True)
fig = px.line(variations_ssum.iloc[15:,:],x=variations_ssum.iloc[15:,:].index,y=variations_ssum.columns.tolist())
fig.update_layout(
    title="Sumatoria de variacion porcentual de los sectores en el SP500",
    xaxis_title="Años",
    yaxis_title="Σ Variación porcentual",
    legend_title="Sector")
fig.show()


En el gráfico anterior se muestra como las empresas del tipo tecnológico-informático son las que mayor variación positiva acumulan, especialmente, se observa como se eleva y se mantiene el sector durante la pandemia del coronavirus (2019).

## ¿Cuáles fueron los momentos de alta volatilidad ?

In [49]:
volatility_grouped = pd.DataFrame(sp500.groupby("Year")["Volatility"].mean())
volatility_grouped.sort_index(inplace=True)
volatility_grouped
py.init_notebook_mode(connected=True)
fig = px.line(volatility_grouped.iloc[1:,:], x=volatility_grouped.iloc[1:,:].index,
              y="Volatility",title="Volatilidad promedio del SP500")
fig.show()

In [50]:
py.init_notebook_mode(connected=True)
fig = px.line(volatility_ssum.iloc[17:,:],x=volatility_ssum.iloc[17:,:].index,y=volatility_ssum.columns.tolist())
fig.update_layout(
    title="Volatilidad en la pandemia del coronavirus",
    xaxis_title="Años",
    yaxis_title="Volatilidad",
    legend_title="Sector")
fig.show()


In [51]:
py.init_notebook_mode(connected=True)
fig = px.line(volatility_ssum.iloc[5:12:,:],x=volatility_ssum.iloc[5:12:,:].index,y=volatility_ssum.columns.tolist())
fig.update_layout(
    title="Volatilidad durante la gran recesión",
    xaxis_title="Años",
    yaxis_title="Volatilidad",
    legend_title="Sector")
fig.show()


## Cuales son las mejores 9 empresas para invertir?

In [52]:

Best_companies_to_invest =pd.DataFrame(sp500.groupby("Ticker")["Variations"].sum())
Best_companies_to_invest.sort_values(by="Variations", ascending=False,inplace=True)
Best_companies_to_invest = Best_companies_to_invest.head(10)

py.init_notebook_mode(connected=True)
fig = px.bar(Best_companies_to_invest,
             x=Best_companies_to_invest.index,
             y="Variations", color='Variations',
            color_continuous_scale=px.colors.sequential.deep,
            title="Empresas con mayor crecimiento")
fig.update_layout(xaxis_title="Compañias")
fig.show()

La variación porcentual acumulada situa a estas compañias como las que mayor crecimiento han tenido en los últimos 20 años, la era de la información explica porque empresas como NVDA o NFLX se situan en este top.